# Notebook for plotting gravity and temperature of Hornindals style beer
This notebook is for plotting the gravity and temperature of beers made using kveik from Hornindal

First the necessary libraries should be loaded.

In [10]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import plotly.io as pio

To be able to do the plotting efficiently for many beers to come I will define a function for reading and modifying the data

In [2]:
def fahr_to_celsius(temp_fahr):
    """Convert Fahrenheit to Celsius
    
    Return Celsius conversion of input"""
    temp_celsius = (temp_fahr - 32) * 5 / 9
    return temp_celsius

In [3]:
def read_tilt_cellphone_data(fname):
    """Read the data from the tilt and return data frame"""
    df = pd.read_csv(fname)
    df.loc[:, "Temp"] = fahr_to_celsius(df["Temp"])
    df.loc[:, "Timepoint"] = pd.to_datetime(df["Timestamp"], format='%d.%m.%Y %H:%M:%S')
    df.set_index("Timepoint", inplace=True)
    df.dropna(subset=["Temp"], inplace=True)
    return df

In [4]:
df = read_tilt_cellphone_data("../data/Hornindal.csv")

In [5]:
df

,Timestamp,SG,Temp,Color,Beer,Comment
Timepoint,,,,,,
2020-09-07 12:07:48,7.9.2020 12:07:48,1.069,35.555556,PURPLE,Vosss,NaN
2020-09-07 12:56:03,7.9.2020 12:56:03,1.065,35.555556,PURPLE,Vosss,NaN
2020-09-07 13:38:24,7.9.2020 13:38:24,1.068,35.555556,PURPLE,Vosss,NaN
2020-09-07 15:11:22,7.9.2020 15:11:22,1.062,36.111111,PURPLE,Vosss,NaN
2020-09-07 15:36:12,7.9.2020 15:36:12,1.059,36.111111,PURPLE,Vosss,NaN
...,...,...,...,...,...,...
2020-09-25 00:27:24,25.9.2020 00:27:24,1.033,35.000000,PURPLE,Hornindal,NaN
2020-09-25 07:43:04,25.9.2020 07:43:04,1.032,34.444444,PURPLE,Hornindal,NaN
2020-09-25 16:56:32,25.9.2020 16:56:32,1.030,34.444444,PURPLE,Hornindal,NaN


Somehow lots of old data ended up in the file. Those should be deleted. There are also some entries with too large temperature and too low SG to be reomved.

In [6]:
df=df.loc[df.index>datetime(2020,9,20)]
df = df.loc[df["Temp"]<40]
df = df.loc[df["SG"]>1.029]
df.index= (df.index -df.index[0]).total_seconds()/3600

In [12]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df.index, y=df["Temp"], name="SG"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df.index, y=df["SG"], name="Temperature"),
    secondary_y=True,
)
fig.show()
pio.write_html(fig, file="index.html", auto_open=True)